## Model Training and Initial Development

This Kaggle competition requires that submissions be made by running code and creating an output file from a Kaggle notebook.
I started by running a copy of the recommended tutorial notebook on Kaggle (https://www.kaggle.com/code/amyjang/monet-cyclegan-tutorial/notebook).

I let the notebook run to completion in an interactive session and tried to submit the result. At that point I learned that the run had taken 569 minutes, using a TPU, and that a maximum of 300 minutes run time was allowed for submission. I noted that the model had trained for 25 epochs. My second run was for 11 epochs, but again it ran too long at 307 minutes. My third run with 10 epochs ran in 300 minutes, and I was able to submit the result. This run produced a Kaggle score of 78.87916.

I then investigated some of the other public notebooks for this competition, looking for improvement suggestions, including this one: https://www.kaggle.com/code/victorsullivan/i-m-something-of-a-painter-myself/notebook .
I noted that the tutorial notebook used a batch size of one, and that many of the public notebooks were using a batch size of 32. Also the tutorial notebook was using tf.data.Dataset.zip to combine a sequence of 300 Monet images with a sequence of 7038 photos, and then training for a fixed number of epochs as in the following block of code.

In [ ]:
cycle_gan_model.fit(
    tf.data.Dataset.zip((monet_ds, photo_ds)),
    epochs=25
)

Zipping the two data sets as above means that the training data was always the 300 Monet images and only the first 300 photos, repeated for every epoch. To use repeating data sets instead, I borrowed the following blocks of code from the VictorSullivan notebook:

In [ ]:
BATCH_SIZE = 32

def get_gan_dataset(monet_files, photo_files, augment=None, repeat=True, shuffle=True, batch_size=1):

    monet_ds = load_dataset(monet_files)
    photo_ds = load_dataset(photo_files)
    
    if augment:
        monet_ds = monet_ds.map(augment, num_parallel_calls=AUTOTUNE)
        photo_ds = photo_ds.map(augment, num_parallel_calls=AUTOTUNE)

    if repeat:
        monet_ds = monet_ds.repeat()
        photo_ds = photo_ds.repeat()
        
#     if shuffle:
#         monet_ds = monet_ds.shuffle(2048)
#         photo_ds = photo_ds.shuffle(2048)
        
    monet_ds = monet_ds.batch(batch_size, drop_remainder=True)
    photo_ds = photo_ds.batch(batch_size, drop_remainder=True)
#     monet_ds = monet_ds.cache()
#     photo_ds = photo_ds.cache()
    monet_ds = monet_ds.prefetch(AUTOTUNE)
    photo_ds = photo_ds.prefetch(AUTOTUNE)
    
    gan_ds = tf.data.Dataset.zip((monet_ds, photo_ds))
    
    return gan_ds

final_data_set = get_gan_dataset(MONET_FILENAMES, PHOTO_FILENAMES, augment=None, 
                                 repeat=True, shuffle=True, batch_size=BATCH_SIZE)

Using the final_data_set for training this way means that the Monet images and photos will each be repeated indefinitely as required. When using a repeating data set this way, the step_per_epoch parameter is needed to determine how many batches are processesed for each epoch of training -- as in this example:

In [ ]:
cycle_gan_model.fit(final_data_set, epochs=28, steps_per_epoch=1490)

With these changes, I ran the model for 30 epochs of 1500 batches each, and got an improved Kaggle score of 40.66223 and a run time of 137 minutes.

I think the improvement in score resulted from 2 factors:
    
    * using the full set of photos instead of just the first 300
    * effectively a lot more training time was availble because a batch size of 32 makes much more efficent use of the TPU versus a batch size of one.